# Save the Fault data for Walanae and Flores

In [1]:
import numpy as np
import pandas as pd
from tsunamibayes.gaussian_process_regressor import GPR
from tsunamibayes.fault import ReferenceCurveFault, GaussianProcessFault

In [3]:
model_bounds = {
    'lat_min': -10,
    'lat_max': -3,
    'lon_min': 112,
    'lon_max': 124
}

## Start with Flores 

In [ ]:
from sensitivity_data_loader import load_data

lats, lons, depths, dips, strikes, rakes = load_data()

np.savez('flores_fault_data', lats=lats, lons=lons, depths=depths, dips=dips, strikes=strikes)#file to save to; should have rakes

In [ ]:
# Load the data to make sure that saving it worked.
ker = lambda x,y: GPR.rbf_kernel(x,y,sig=0.75)
flores_data = np.load('flores_fault_data.npz')
fault = GaussianProcessFault( # The Flores fault uses a GaussianProcessFault
    bounds=model_bounds,
    kers={'depth': ker, 'dip': ker, 'strike': ker},
    noise={'depth': 1, 'dip': 1, 'strike': 1},
    **flores_data
)
fault

## Now save the Walanae data

In [9]:
import pickle
# Get latitude and longitude of faults.

use_hypo = False # I think the Hypo data is wrong -- everything seems shifted too far left on the Walanae fault.
if not use_hypo:
    flores = pd.read_excel('1820_fault_data/FloresThrustSplit_TableToExcel.xlsx')
    walanae = pd.read_excel('1820_fault_data/WalinaeFaultSplit_TableToExcel.xlsx')
    flores_lats, flores_lons = flores[['CenterLat', 'CenterLong']].to_numpy().T
    walanae_lats, walanae_lons = walanae[['CenterLat', 'CenterLong']].to_numpy().T
else:
    flores = pd.read_excel('1820_fault_data/FloresHypoSegments.xlsx')
    walanae = pd.read_excel('1820_fault_data/WalanaeHypoSegments.xlsx')
    flores_lats, flores_lons = flores[['Lat', 'Long']].to_numpy().T
    walanae_lats, walanae_lons = walanae[['Lat', 'Long']].to_numpy().T
    

# Get the strike, dip, and depth data needed to initialize a ReferenceCurveFault.
walanae_strikes = walanae['Strike'].to_numpy()

# Dip angle assumed to be 25 degrees.
def walanae_dip(x):
    import numpy as np
    return np.ones(np.shape(x))*25 

# Depths are assumed to be 20 km.
def walanae_depth(dist):
    """Gives depth based on distance from fault. 
    A negative distance is higher than the base fault depth.
    """
    import numpy as np
    base_depth = 20000
    extra_depth = dist*np.tan(np.deg2rad(walanae_dip(dist)))
    return base_depth - extra_depth

to_save = {
    'latpts': walanae_lats,
    'lonpts': walanae_lons,
    'strikepts': walanae_strikes,
    'depth_curve': walanae_depth,
    'dip_curve': walanae_dip
}
with open('walanae_fault_data.pkl', 'wb') as walanae_file:
    pickle.dump(to_save, walanae_file)

In [10]:
# Load the data to make sure it worked.
walanae_data = pickle.load()
fault = ReferenceCurveFault(bounds=model_bounds, **walanae_data)
fault

TypeError: file must have 'read' and 'readline' attributes

In [5]:
np.load?

In [6]:
np.savez?